<a href="https://colab.research.google.com/github/ikoryakovskiy/machine_learning/blob/master/multi_grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.layer = Dense(2, kernel_initializer=tf.constant_initializer(1))

        # add weight actually belongs to a layer
        self.alpha = self.add_weight(
            "alpha", shape=[2, 2], trainable=True, initializer=tf.constant_initializer(-1)
        )

    def call(self, x):
        y0 = self.layer(x)
        y1 = tf.matmul(x, self.alpha)
        z = tf.math.accumulate_n([y0, y1])
        return tf.math.reduce_sum(z)


In [ ]:
#input = tf.keras.Input(shape=(None, 2))
model = MyModel()
model.build((None, 2))
model.compile(optimizer='sgd', loss='mse', metrics='mse')
model.summary()

Model: "my_model_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             multiple                  6         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [ ]:
input = tf.keras.Input(shape=(2,))
# input = tf.Variable([[1.0, 1.0]])  # this is equivalent to the previous
model(input)
print(model.input)
print(model.output)

Tensor("input_68:0", shape=(None, 2), dtype=float32)
Tensor("my_model_48/Sum:0", shape=(), dtype=float32)


In [ ]:
alphas, weights = [], []
for tensor in model.trainable_variables:
    if "alpha" in tensor.name:
        alphas.append(tensor)
    else:
        weights.append(tensor)
print(alphas)
print(weights)

[<tf.Variable 'alpha:0' shape=(2, 2) dtype=float32, numpy=
array([[-1., -1.],
       [-1., -1.]], dtype=float32)>]
[<tf.Variable 'dense_59/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>, <tf.Variable 'dense_59/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]


In [ ]:
def apply(vars, grads, lr):
    for var, grad in zip(vars, grads):
      var.assign_sub(grad * lr)

In [ ]:
input = tf.Variable([[1.0, 1.0]])
real_y = [4]

with tf.GradientTape() as tape:
    pred_y = model(input)
    model_loss = tf.keras.losses.mse(real_y, pred_y)
alpha_grads, weight_grads = tape.gradient(model_loss, (alphas, weights))

#print(alpha_grads)
#print(weight_grads)

print(alphas)
print(weights)

apply(alphas, alpha_grads, 0.001)
apply(weights, weight_grad, 0.05)

print(alphas)
print(weights)

[<tf.Variable 'alpha:0' shape=(2, 2) dtype=float32, numpy=
array([[-1., -1.],
       [-1., -1.]], dtype=float32)>]
[<tf.Variable 'dense_59/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>, <tf.Variable 'dense_59/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]
[<tf.Variable 'alpha:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.992, -0.992],
       [-0.992, -0.992]], dtype=float32)>]
[<tf.Variable 'dense_59/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[1.5, 1.5],
       [1.5, 1.5]], dtype=float32)>, <tf.Variable 'dense_59/bias:0' shape=(2,) dtype=float32, numpy=array([0.5, 0.5], dtype=float32)>]
